# Imports and Setups

In [1]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 11.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 110.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 83.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 3.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


Lendo o Arquivo

In [1]:
from google.colab import drive
drive.mount('drive')

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [2]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas()


In [3]:
num_gen = 5
perc = 5
i=5

In [4]:
arq = '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/emoint/base_emoint_'+str(perc)+'_original'+str(i)+'.csv'
base_cru_1p = pd.read_csv(arq, encoding='utf-8', sep=';', decimal = ',')

In [5]:
base_cru_1p.head()

,texto,label
0,"Bloody hell Pam, calm yourself down. But could...",0
1,People who cheer for sports teams completely o...,0
2,@user awe thanks girl 😊😊,1
3,Watching Avatar and wondering why I took so lo...,1
4,@user @user Donald Trump in the White House #s...,3


In [6]:
def cria_prompt(ordem,synt):
  prompt = ordem

  prompt += 'Tweet: '+ synt+'\nResult: '
  return prompt

In [7]:
inicio = """Rewrite the following tweets 5 times to generate 5 different paraphrases  from original tweet.
Return in Python list like this ["paraphrase1","paraphrase2","paraphrase3","paraphrase4","paraphrase5"] as example.

Tweet:What are the best places to see in New York?
Result:["What are some must-see places in New York?","Can you suggest some must-see spots in New York?","Where should one go to experience the best NYC has to offer?","Which places should I visit in New York?","What are the top destinations to explore in New York?"]
###
"""
base_cru_1p['prompt'] = base_cru_1p.progress_apply(lambda x: cria_prompt(inicio, x.texto), axis=1)
base_cru_1p.head()

100%|██████████| 163/163 [00:00<00:00, 61481.25it/s]


,texto,label,prompt
0,"Bloody hell Pam, calm yourself down. But could...",0,Rewrite the following tweets 5 times to genera...
1,People who cheer for sports teams completely o...,0,Rewrite the following tweets 5 times to genera...
2,@user awe thanks girl 😊😊,1,Rewrite the following tweets 5 times to genera...
3,Watching Avatar and wondering why I took so lo...,1,Rewrite the following tweets 5 times to genera...
4,@user @user Donald Trump in the White House #s...,3,Rewrite the following tweets 5 times to genera...


In [8]:
base_cru_1p.texto.iloc[90]

'@user I love parody accounts! Well done. Vote for #Trump. #lol'

In [9]:
print(base_cru_1p.prompt.iloc[90])

Rewrite the following tweets 5 times to generate 5 different paraphrases  from original tweet.
Return in Python list like this ["paraphrase1","paraphrase2","paraphrase3","paraphrase4","paraphrase5"] as example.

Tweet:What are the best places to see in New York?
Result:["What are some must-see places in New York?","Can you suggest some must-see spots in New York?","Where should one go to experience the best NYC has to offer?","Which places should I visit in New York?","What are the top destinations to explore in New York?"]
###
Tweet: @user I love parody accounts! Well done. Vote for #Trump. #lol
Result: 


In [10]:
n_labels = base_cru_1p.label.nunique()
n_labels

4

In [11]:
! huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) y
Token is valid (permission: read).
Cannot authenticate through git-cr

In [12]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "meta-llama/Llama-2-7b-chat-hf"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model_4bit = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [13]:
prompt = """Rewrite the following tweets 5 times to generate 5 different paraphrases  from original tweet.
Return in Python list like this ["paraphrase1","paraphrase2","paraphrase3","paraphrase4","paraphrase5"] as example.

Tweet:What are the best places to see in New York?
Result:["What are some must-see places in New York?","Can you suggest some must-see spots in New York?","Where should one go to experience the best NYC has to offer?","Which places should I visit in New York?","What are the top destinations to explore in New York?"]
###
Tweet: The Quarterback' wrecks me every time..
Result: """

In [14]:
def gera_text(modelo, tokens, dev, ppt, temperatura, max_tokens):
  inputs = tokens(ppt, return_tensors="pt").to(dev)
  outputs = modelo.generate(**inputs, max_new_tokens=max_tokens, temperature=temperatura, do_sample =True)
  result = tokenizer.decode(outputs[0], skip_special_tokens=True)
  return result

In [15]:
device = "cuda:0"
print(gera_text(model_4bit, tokenizer, device, prompt, 0.1, 100))

Rewrite the following tweets 5 times to generate 5 different paraphrases  from original tweet.
Return in Python list like this ["paraphrase1","paraphrase2","paraphrase3","paraphrase4","paraphrase5"] as example.

Tweet:What are the best places to see in New York?
Result:["What are some must-see places in New York?","Can you suggest some must-see spots in New York?","Where should one go to experience the best NYC has to offer?","Which places should I visit in New York?","What are the top destinations to explore in New York?"]
###
Tweet: The Quarterback' wrecks me every time..
Result:  ["The Quarterback's impact never fails to move me...", "The Quarterback's brilliance always leaves me in awe...", "The Quarterback's talent is unmatched and always brings me to tears...", "The Quarterback's greatness never ceases to amaze me...", "The Quarterback's talent is unparalleled and always leaves me breathless..."]










In [16]:
base_cru_1p['generated'] = base_cru_1p['prompt'].progress_apply(lambda x: gera_text(model_4bit, tokenizer, device, x, 0.1, 220))
base_cru_1p.head()

100%|██████████| 163/163 [54:30<00:00, 20.06s/it]


,texto,label,prompt,generated
0,"Bloody hell Pam, calm yourself down. But could...",0,Rewrite the following tweets 5 times to genera...,Rewrite the following tweets 5 times to genera...
1,People who cheer for sports teams completely o...,0,Rewrite the following tweets 5 times to genera...,Rewrite the following tweets 5 times to genera...
2,@user awe thanks girl 😊😊,1,Rewrite the following tweets 5 times to genera...,Rewrite the following tweets 5 times to genera...
3,Watching Avatar and wondering why I took so lo...,1,Rewrite the following tweets 5 times to genera...,Rewrite the following tweets 5 times to genera...
4,@user @user Donald Trump in the White House #s...,3,Rewrite the following tweets 5 times to genera...,Rewrite the following tweets 5 times to genera...


In [17]:
base_cru_1p.head()

,texto,label,prompt,generated
0,"Bloody hell Pam, calm yourself down. But could...",0,Rewrite the following tweets 5 times to genera...,Rewrite the following tweets 5 times to genera...
1,People who cheer for sports teams completely o...,0,Rewrite the following tweets 5 times to genera...,Rewrite the following tweets 5 times to genera...
2,@user awe thanks girl 😊😊,1,Rewrite the following tweets 5 times to genera...,Rewrite the following tweets 5 times to genera...
3,Watching Avatar and wondering why I took so lo...,1,Rewrite the following tweets 5 times to genera...,Rewrite the following tweets 5 times to genera...
4,@user @user Donald Trump in the White House #s...,3,Rewrite the following tweets 5 times to genera...,Rewrite the following tweets 5 times to genera...


In [18]:
import ast
def transforme_list(x):
  try:
    resp = ast.literal_eval(x.split("Result:")[2].replace("\n", "").strip())
  except:
    resp = []
  return resp

In [19]:
print(base_cru_1p.generated.iloc[4])

Rewrite the following tweets 5 times to generate 5 different paraphrases  from original tweet.
Return in Python list like this ["paraphrase1","paraphrase2","paraphrase3","paraphrase4","paraphrase5"] as example.

Tweet:What are the best places to see in New York?
Result:["What are some must-see places in New York?","Can you suggest some must-see spots in New York?","Where should one go to experience the best NYC has to offer?","Which places should I visit in New York?","What are the top destinations to explore in New York?"]
###
Tweet: @user @user Donald Trump in the White House #shudder
Result:  ["@user Donald Trump in the White House #shudder","@user Donald Trump in the White House #facepalm","@user Donald Trump in the White House #disgusted","@user Donald Trump in the White House #sadness","@user Donald Trump in the White House #fear"]
























































































































































In [20]:
base_cru_1p.generated.iloc[0].split("Result:")[2].replace("\n", "").strip()

'["Bloody hell Pam, calm down! But I swear I just saw something black and hairy scurry across the carpet. #perils of living alone", "Pam, take a deep breath. I think I just saw something furry and dark cross the carpet. #livingalone #fears", "Pam, try to remain composed. I\'m pretty sure I just saw a black and hairy creature dart across the carpet. #alone #frightened", "Pam, don\'t panic. I think I just saw something dark and furry scurry across the carpet. #livingalone #anxiety", "Pam, try to stay calm. I\'m pretty sure I just saw a black and hairy creature run across the carpet. #fears #alone"]'

In [21]:
print(ast.literal_eval(base_cru_1p.generated.iloc[3].split("Result:")[2].replace("\n", "")))

['Watching Avatar and feeling overwhelmed by its greatness *cheers*', 'Avatar viewing experience leaves me in awe *fist pump*', "Finally watched Avatar and I'm in movie heaven *swoon*", 'Why did I wait so long to watch Avatar? *head desk*', 'Avatar viewing has left me in a state of pure bliss *heart eyes*']


In [22]:
base_cru_1p['generated'] = base_cru_1p['generated'].progress_apply(lambda x: transforme_list(x))

100%|██████████| 163/163 [00:00<00:00, 16450.63it/s]


In [23]:
base_cru_1p.head()

,texto,label,prompt,generated
0,"Bloody hell Pam, calm yourself down. But could...",0,Rewrite the following tweets 5 times to genera...,"[Bloody hell Pam, calm down! But I swear I jus..."
1,People who cheer for sports teams completely o...,0,Rewrite the following tweets 5 times to genera...,[Those who cheer for sports teams beyond NJ or...
2,@user awe thanks girl 😊😊,1,Rewrite the following tweets 5 times to genera...,"[Thanks girl 😊😊, Girl, you're awesome! 😊😊, Awe..."
3,Watching Avatar and wondering why I took so lo...,1,Rewrite the following tweets 5 times to genera...,[Watching Avatar and feeling overwhelmed by it...
4,@user @user Donald Trump in the White House #s...,3,Rewrite the following tweets 5 times to genera...,[@user Donald Trump in the White House #shudde...


In [24]:
export = base_cru_1p.copy()
export = export.explode('generated').reset_index(drop=True)


In [25]:
export.head()

,texto,label,prompt,generated
0,"Bloody hell Pam, calm yourself down. But could...",0,Rewrite the following tweets 5 times to genera...,"Bloody hell Pam, calm down! But I swear I just..."
1,"Bloody hell Pam, calm yourself down. But could...",0,Rewrite the following tweets 5 times to genera...,"Pam, take a deep breath. I think I just saw so..."
2,"Bloody hell Pam, calm yourself down. But could...",0,Rewrite the following tweets 5 times to genera...,"Pam, try to remain composed. I'm pretty sure I..."
3,"Bloody hell Pam, calm yourself down. But could...",0,Rewrite the following tweets 5 times to genera...,"Pam, don't panic. I think I just saw something..."
4,"Bloody hell Pam, calm yourself down. But could...",0,Rewrite the following tweets 5 times to genera...,"Pam, try to stay calm. I'm pretty sure I just ..."


In [26]:
export.shape

(783, 4)

In [27]:
base_cru_1p.shape

(163, 4)

In [28]:
export.shape[0]/base_cru_1p.shape[0]

4.803680981595092

In [29]:
arquivo = '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/emoint/gptparaphrase/base_emoint_'+str(perc)+'_gptparaphrase'+str(i)+'.pkl'
export[['texto', 'label', 'generated']].to_pickle(arquivo)